In [135]:
import numpy as np
import tensorflow as tf
from functools import reduce
import matplotlib.pyplot as plt
sess = tf.InteractiveSession()

In [136]:
num_features = 2
num_samples = 5
state_size = 4
num_features = 4
num_of_operations = 3
batch_size = num_samples
max_num_features = 10

def add(vec):
    return reduce((lambda x, y: x + y),vec)

def samples_generator(fn, shape, rng, seed=None):
    '''
    Generate random samples for the model:
    @fn - function to be applied on the input features to get the ouput
    @shape - shape of the features matrix (num_samples, num_features)
    @rng - range of the input features to be generated within (a,b)
    @seed  - generation seed
    Outputs a tuple of input and output features matrix
    '''
    x = (rng[1] - rng[0]) * np.random.random_sample(shape) + rng[0]
    y = np.apply_along_axis(fn, 1, x).reshape((-1,1))

    #x = x.reshape((batch_size, -1))  # The first index changing slowest, subseries as rows
    #y = y.reshape((batch_size, -1))

    return (x, y)


x,y = samples_generator(add, (num_samples, num_features) , (0, 100))

def tf_multiply(inpt):
    return tf.reshape( tf.reduce_prod(inpt, axis = 1), [batch_size, -1])

def tf_add(inpt):
    return  tf.reshape( tf.reduce_sum(inpt, axis = 1), [batch_size, -1])

def tf_stall(a):
    return a


x = tf.Variable(x, dtype=tf.float32)

x.initializer.run()



In [137]:
add    = tf_add(x)
mult   = tf_multiply(x)
stall  = tf_stall(x)
values = tf.concat([add, mult, stall], 1)
print(values.eval())

[[  2.49476181e+02   1.19012610e+07   3.84333611e+01   9.29976349e+01
    7.14285660e+01   4.66166191e+01]
 [  1.81322830e+02   4.11986550e+06   4.27255707e+01   5.26204147e+01
    4.69276009e+01   3.90492363e+01]
 [  1.57355591e+02   1.12406631e+04   5.37128487e+01   3.58061447e+01
    6.77503128e+01   8.62670392e-02]
 [  2.58221954e+02   1.65438550e+07   7.43373108e+01   6.39807472e+01
    7.07143021e+01   4.91895981e+01]
 [  2.93363800e+02   2.79430180e+07   8.07342987e+01   7.54765930e+01
    7.93957977e+01   5.77571068e+01]]


In [226]:
init_state = tf.Variable(np.zeros([batch_size, state_size]), dtype=tf.float32)
argmax_dum = tf.Variable([2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],  dtype=tf.int32)
W = tf.Variable(np.random.rand(state_size+num_features, state_size), dtype=tf.float32)
b = tf.Variable(np.zeros((state_size)), dtype=tf.float32)

W2 = tf.Variable(np.random.rand(state_size, num_of_operations),dtype=tf.float32)
b2 = tf.Variable(np.zeros((num_of_operations)), dtype=tf.float32)

dummy_matrix = tf.zeros([batch_size, num_features], tf.float32, name="dummy_constant")

init_state.initializer.run()
argmax_dum.initializer.run()
W.initializer.run()
b.initializer.run()
W2.initializer.run()
b2.initializer.run()
#dummy_matrix.initializer.run()

current_state = init_state
current_input = x
      
input_and_state_concatenated = tf.concat([current_input, current_state], 1)  # Increasing number of columns
next_state = tf.tanh(tf.matmul(input_and_state_concatenated, W) + b)  # Broadcasted addition
current_state = next_state
    
    #calculate softmax and produce the mask of operations
logits  = tf.matmul(next_state, W2) + b2 #Broadcasted addition
softmax = tf.nn.softmax(logits)
softmax_10000 = tf.nn.softmax(logits)
softplus = tf.nn.softplus(logits)
softsign = tf.nn.softsign(logits)
argmax  = tf.argmax(softmax, 1)

#lengths_transposed = tf.expand_dims(argmax, 0)
#onehot  = tf.one_hot(argmax, num_of_operations)
stall_width = tf.shape(stall)[1] - 1 
stall_select = tf.slice(softmax, [0,2], [batch_size,1])
multiples = [1, stall_width]
stall_mask = tf.tile(stall_select, multiples)
mask = tf.concat([softmax, stall_mask], 1)
#mask = tf.cast(mask, tf.bool)
#apply mask
#output = tf.boolean_mask(values,mask)
output = values * mask
nn_soft = tf.nn.softmax(logits)
output_width = tf.shape(output)[1]
zeros_slice = tf.slice(dummy_matrix, [0,0], [batch_size, max_num_features - output_width])
dum_plus_input = tf.concat([output, zeros_slice], 1)
print(dum_plus_input.eval())

'''
print(x.eval())
print("stall_select")
print(stall_select.eval())
print("mask")
print(mask.eval())
print("logits")
print(logits.eval())
print("softmax")
print(softmax.eval())
print("softplus")
print(softplus.eval())
print("softsign")
print(softsign.eval())
print("softmax*10000")
print(softmax_10000.eval())
print("onehot")
print(onehot.eval())
'''
print(mask.eval())
#print(mask.eval())
#print(output.eval())

[[  6.13667564e+01   5.42079300e+06   1.14737835e+01   2.77632427e+01
    2.13240757e+01   1.39167891e+01   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  4.46022301e+01   1.87651862e+06   1.27551670e+01   1.57091446e+01
    1.40096283e+01   1.16576452e+01   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  3.87067108e+01   5.11990332e+03   1.60352764e+01   1.06894617e+01
    2.02259808e+01   2.57539097e-02   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  6.35180626e+01   7.53540400e+06   2.21924438e+01   1.91006260e+01
    2.11108418e+01   1.46849184e+01   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  7.21623383e+01   1.27275010e+07   2.41021805e+01   2.25325623e+01
    2.37025890e+01   1.72426376e+01   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]
[[ 0.24598242  0.45548055  0.29853708  0.29853708  0.29853708  0.29853708]
 [ 0.24598242  0.45548055  0.29853708  

In [227]:
add_softmax   = tf.slice(softmax, [0,0], [batch_size,1])
mult_softmax  = tf.slice(softmax, [0,1], [batch_size,1])
stall_softmax = tf.slice(softmax, [0,2], [batch_size,1])

add_width   = tf.shape(add)[1]
mult_width  = tf.shape(mult)[1]
stall_width = tf.shape(stall)[1]


add_final   = add * add_softmax
mult_final  = mult * mult_softmax
stall_final = stall * stall_softmax

##conact add and mult results with zeros matrix
add_final = tf.concat([add_final, tf.slice(dummy_matrix, [0,0], [batch_size, num_features - add_width])], 1) 
mult_final = tf.concat([mult_final, tf.slice(dummy_matrix, [0,0], [batch_size, num_features - mult_width])], 1) 

output_final = add_final + mult_final + stall_final 

print(add_width.eval())
print(mult_width.eval())
print(stall_width.eval())


print(add.eval())
print(mult.eval())
print(stall.eval())

print(add_softmax.eval())
print(mult_softmax.eval())
print(stall_softmax.eval())

print(add_final.eval())
print(mult_final.eval())
print(stall_final.eval())

print(output_final.eval())

1
1
4
[[ 249.47618103]
 [ 181.3228302 ]
 [ 157.35559082]
 [ 258.22195435]
 [ 293.36380005]]
[[  1.19012610e+07]
 [  4.11986550e+06]
 [  1.12406631e+04]
 [  1.65438550e+07]
 [  2.79430180e+07]]
[[  3.84333611e+01   9.29976349e+01   7.14285660e+01   4.66166191e+01]
 [  4.27255707e+01   5.26204147e+01   4.69276009e+01   3.90492363e+01]
 [  5.37128487e+01   3.58061447e+01   6.77503128e+01   8.62670392e-02]
 [  7.43373108e+01   6.39807472e+01   7.07143021e+01   4.91895981e+01]
 [  8.07342987e+01   7.54765930e+01   7.93957977e+01   5.77571068e+01]]
[[ 0.24598242]
 [ 0.24598242]
 [ 0.24598242]
 [ 0.24598242]
 [ 0.24598242]]
[[ 0.45548055]
 [ 0.45548055]
 [ 0.45548055]
 [ 0.45548055]
 [ 0.45548055]]
[[ 0.29853708]
 [ 0.29853708]
 [ 0.29853708]
 [ 0.29853708]
 [ 0.29853708]]
[[ 61.36675644   0.           0.           0.        ]
 [ 44.60223007   0.           0.           0.        ]
 [ 38.70671082   0.           0.           0.        ]
 [ 63.51806259   0.           0.           0.        ]
 [ 

In [142]:

def custom_softmax(inpt):
    sum_exponents = tf.reduce_sum(tf.exp(inpt), axis = 1)
    soft_max = tf.log(sum_exponents)
    soft_max = tf.reshape( soft_max, [batch_size, -1])
    print(soft_max.eval())
    matrix_width = tf.cast(tf.shape(inpt)[1], tf.int32)
    multiples = [1, matrix_width]
    soft_max = tf.tile(soft_max, multiples)
    return  tf.reshape( inpt / soft_max, [batch_size, -1])

def custom_softmax2(inpt):
    sum_exponents = tf.reduce_sum(tf.exp(inpt), axis = 1)
    soft_max = sum_exponents
    soft_max = tf.reshape( soft_max, [batch_size, -1])
    print(soft_max.eval())
    matrix_width = tf.cast(tf.shape(inpt)[1], tf.int32)
    multiples = [1, matrix_width]
    soft_max = tf.tile(soft_max, multiples)
    return  tf.reshape( tf.exp(inpt) / soft_max, [batch_size, -1])
x_cust = custom_softmax(x)
x_cust2 = custom_softmax2(x)
print(x.eval())
#print(x_cust2.eval())
print(tf.nn.softmax(x*10000).eval())

[[         inf]
 [ 52.62382889]
 [ 67.75031281]
 [ 74.36369324]
 [ 80.97130585]]
[[             inf]
 [  7.14890146e+22]
 [  2.65208442e+29]
 [  1.97579466e+32]
 [  1.46349014e+35]]
[[  3.84333611e+01   9.29976349e+01   7.14285660e+01   4.66166191e+01]
 [  4.27255707e+01   5.26204147e+01   4.69276009e+01   3.90492363e+01]
 [  5.37128487e+01   3.58061447e+01   6.77503128e+01   8.62670392e-02]
 [  7.43373108e+01   6.39807472e+01   7.07143021e+01   4.91895981e+01]
 [  8.07342987e+01   7.54765930e+01   7.93957977e+01   5.77571068e+01]]
[[ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]]


In [27]:
sess.close()